# Advanced APIs

### Set up for Success

In [ ]:
import requests

url = 'https://cae-bootstore.herokuapp.com'

endpoint_login = "/login"
endpoint_user = "/user"
endpoint_book = "/book"



## GET Requests

In [ ]:
def get_books():
    books = requests.get(url+endpoint_book)
    return books.json()['books']
books = get_books()
books

## In Class Assignment A

#### make a function that returns a set of all the categories/subject found in the book list

In [ ]:
def get_category_list(books):
    return {book['subject'].title() for book in books}
get_category_list(books)

## In Class Assignment B

#### make a function that returns a list of all the books(dicts) in a category

In [4]:
def get_book_by_category(books, category):
    return list(filter(lambda book: book['subject'].title()==category.title(),books))
get_book_by_category(books,'Cooking')

[{'author': 'Anonymous',
  'created_on': 'Mon, 28 Feb 2022 19:21:17 GMT',
  'id': 23,
  'img': 'https://s2982.pcdn.co/wp-content/uploads/2018/11/cooking-to-kill-book-cover.jpg',
  'pages': 3207,
  'subject': 'cooking',
  'summary': 'Originally titled: Divorce with Benefits',
  'title': 'COOKING TO KILL: THE POISON COOK-BOOK'},
 {'author': 'Malcolm',
  'created_on': 'Wed, 09 Mar 2022 23:12:01 GMT',
  'id': 48,
  'img': 'http://www.fastprint.co.uk/Assets/User/2064-eating-people-is-wrong.jpg',
  'pages': 134,
  'subject': 'cooking',
  'summary': 'This was also news to me',
  'title': 'Eating People is Wrong'},
 {'author': 'Coolio',
  'created_on': 'Wed, 09 Mar 2022 23:12:01 GMT',
  'id': 52,
  'img': 'http://www.fastprint.co.uk/Assets/User/2062-cookin-with-coolio.jpg',
  'pages': 207,
  'subject': 'cooking',
  'summary': 'The fried bologna is devine',
  'title': "Cookin' with Coolio"}]

## Using a SLUG
A Slug is the unique identifying part of a web address, typically at the end of the URL

In [5]:
def get_single_book(book_id):
    single_book = requests.get(url+endpoint_book +'/'+str(book_id))
    return single_book.json()

get_single_book(22)

{'author': 'Micheala Giles',
 'created_on': 'Mon, 28 Feb 2022 19:21:17 GMT',
 'id': 22,
 'img': 'https://s2982.pcdn.co/wp-content/uploads/2018/11/commuter-pig-keeper-book-cover.jpg',
 'pages': 87,
 'subject': 'science',
 'summary': 'Not in Kansas anymore? Raising pigs in the City with a full time job?',
 'title': 'THE COMMUTER PIG KEEPER'}

## POST Requests

#### json.dumps will convert a dictionary into a JSON String ready for transport

In [6]:
import json

def register_user(payload):
    payload_json_string = json.dumps(payload)
    headers = {
        'Content-Type':'application/json'
    }
    response = requests.post(
        url + endpoint_user,
        data = payload_json_string,
        headers = headers
    )
    return response.text
    
    
    
    
jims_payload={
    "email":"jimb@eam.com",
    "first_name":"William",
    "last_name":"Beam",
    "password":"123"
}

register_user(jims_payload)    


'success'

## Authentication - Basic Authorization


In [7]:
import base64

def login_user(user_name, password):
    auth_string = user_name + ":" + password
    
    headers={
        'Authorization' : "Basic "+base64.b64encode(auth_string.encode()).decode()
    }
    
    user_data = requests.get(
        url + endpoint_login,
        headers=headers
    )
    return user_data.json()
login_user('jimb@eam.com','123')
    

{'created_on': 'Thu, 21 Jul 2022 21:23:44 GMT',
 'email': 'jimb@eam.com',
 'first_name': 'William',
 'last_name': 'Beam',
 'modified_on': 'Thu, 21 Jul 2022 21:23:48 GMT',
 'token': 'W5KCh-hh6YLv-4AIu6BFsLcZKNS5664ZFhtiDFI-IzM',
 'user_id': 231}

In [8]:
jim=login_user('jimb@eam.com','123')
print(jim['token'])

W5KCh-hh6YLv-4AIu6BFsLcZKNS5664ZFhtiDFI-IzM


## Authentication - Bearer Authorization & PUT Requests

In [9]:
def edit_user(token, payload):
    payload_json_string = json.dumps(payload)
    headers={
        'Content-Type':'application/json',
        'Authorization':'Bearer ' + token
    }
    response = requests.put(
        url + endpoint_user,
        data=payload_json_string,
        headers=headers
    )
    return response.text

jims_edit_payload={
    "first_name":"Bill"
}

edit_user(jim['token'], jims_edit_payload)


'success'

In [10]:
jim=login_user('jimb@eam.com','123')
jim['first_name']

'Bill'

## DELETE Request

In [ ]:
def delete_user(token):
    headers = {
        'Authorization':"Bearer " + token
    }
    
    response = requests.delete(
        url+endpoint_user,
        headers=headers
    )
    return response.text
print(delete_user(jim['token']))


In [ ]:
jim=login_user('jimb@eam.com','123')


## Display Images in Jupyter notebook

In [13]:
from IPython.display import Image, display
image = Image(url='https://s2982.pcdn.co/wp-content/uploads/2018/11/commuter-pig-keeper-book-cover.jpg')
display(image)
print('hello')

hello


## Make an Application

In [14]:
from getpass import getpass
import time
from IPython.display import clear_output
from IPython.display import Image
from IPython.display import display


def login(email):
    clear_output()
    password=getpass("Password: ")
    user = login_user(email, password) 
    return user

def register():
    clear_output()
    print("Registration:")
    email = input("Email: ")
    first_name = input("First Name: ")
    last_name = input("Last Name: ")
    password = getpass("Password: ")
    
    user_dict={
        "email":email,
        "first_name":first_name,
        "last_name":last_name,
        "password":password
    }
    return register_user(user_dict)

def display_book_short(book):
    print(f"{book['id']} \t| {book['title'][:50].ljust(50)} | \t{book['subject']}")

def display_book_long(book):
    cover = Image(url=book['img'])
    display(cover)
    print(f'''
Title:\t {book['title']}
Author: {book['author']}
Pages:\t {book['pages']}
Subject: {book['subject']}
Summary: {book['summary']}
''')
    

    
class ReadingList():
    def __init__(self):
        self.reading_list=[]
        
    def add_book(self, book):
        if book not in self.reading_list:
            self.reading_list.append(book)
    
    def remove_book(self, book):
        self.reading_list.remove(book)
    
    def empty(self):
        self.reading_list=[]
    
    def show_book_list(self):
        clear_output()
        if not self.reading_list:
            print("Your reading list is empty")
        for book in self.reading_list:
            print(f'''
{"="*50}            
\n
Title:\t {book['title']}
Book ID: {book['id']}
Author:\t {book['author']}
Subject: {book['subject']}
Summary: {book['summary']}
\n
{"="*50}            
\n
''')
    
    def show_small_book_list(self):
        if not self.reading_list:
            print("Your reading list is empty")
        else:
            for book in self.reading_list:
                display_book_short(book)

def browse_books(books, reading_list, subject=None):
    while True:
        clear_output()
        print(f'''
Welcome to the Book Browser
You are viewing {subject if subject else 'all'} books
[ID] \t| {"[TITLE]".ljust(50)} | [SUBJECT]
        ''')
        if subject:
            books=get_book_by_category(books, subject)
        for book in books:
            display_book_short(book)

        selection = input("Select you book by ID [BACK to back out]")
        if selection.lower()=='back':
            return
        elif selection.isnumeric() and int(selection) in map(lambda book: book['id'], books):
            selection=int(selection)
            while True:
                print(f'''
You Selected: {list(filter(lambda book: book['id'] == selection, books))[0]['title']}
1. Add Book To Reading List
2. View More Information
3. Go Back
4. Go To Main Menu
''')
                action = input("Action: ")
                if action =="1":
                    reading_list.add_book(list(filter(lambda book: book['id']==selection, books))[0])
                    print("As you wish")
                    time.sleep(1)
                    break
                elif action =="2":
                    clear_output()
                    display_book_long(list(filter(lambda book: book['id']==selection, books))[0])
                    input("Press Enter To Continue")
                elif action=="3":
                    break
                elif action=="4":
                    return
                    
        else:
            print("Invalid ID")
            time.sleep(2)
            continue
    

def main():
    reading_list = ReadingList()
    books = get_books()
    
    while True:
        clear_output()
        print("Welcome to the Bookstore")
        email = input("Type your email to login or Type `register` to Register ")
        if email == 'register':
            success_register=register()
            if success_register:
                print("You have successfully registered")
                continue
        elif email.lower() == "quit":
            print("Goodbye")
            break
        else:
            try:
                login(email)
            except:
                print("Invalid Username/Password combo")
                time.sleep(2)
                continue
        # First Scene of our app (above)
        while True:
            clear_output()
            print("""
Welcome to the Repository            
You can:            
1. Browse All Books
2. Browse Book by Category
3. View Reading List
4. Remove Book From Reading List
5. Quit     
""")
            command = input("Select your Fate: ")
            if command == "1":
                browse_books(books, reading_list)
            elif command == "2":
                while True:
                    print(" | ".join(get_category_list(books)))
                    cat = input("Category: ").title()
                    if cat in get_category_list(books):
                        browse_books(books, reading_list, cat)
                        break
                    print("Invalid Category")
                    time.sleep(2)
                    continue
            elif command == "3":
                reading_list.show_book_list()
                input("Press Enter To Return")
                continue
            elif command == "4":
                while True:
                    clear_output()
                    reading_list.show_small_book_list()
                    garbage = input("What book ID would you like to remove? [BACK to back out]")
                    if garbage.lower() == "back":
                        break
                    elif garbage.isnumeric() and int(garbage) in map(lambda book: book['id'], reading_list.reading_list):
                        reading_list.remove_book(list(filter(lambda book: book['id']==int(garbage), reading_list.reading_list))[0])
                        print(f'{garbage} has been removed')
                        time.sleep(2)
                        break
                    else:
                        print(f'{garbage} is not in your collection')
                        time.sleep(2)
                        break
                continue   
                    
            elif command == "5":
                print("Goodbye")
                break
            else:
                print("Invalid Selection")
                time.sleep(2)
                continue
        break
            

main()            


Welcome to the Repository            
You can:            
1. Browse All Books
2. Browse Book by Category
3. View Reading List
4. Remove Book From Reading List
5. Quit     

Select your Fate: 5
Goodbye


In [ ]:
cindy@crawford.com

In [ ]:
from getpass import getpass
import time
from IPython.display import clear_output
from IPython.display import Image
from IPython.display import display


def login(email):
    clear_output()
    password=getpass("Password: ")
    user = login_user(email, password) 
    return user

def register():
    clear_output()
    print("Registration:")
    email = input("Email: ")
    first_name = input("First Name: ")
    last_name = input("Last Name: ")
    password = getpass("Password: ")
    
    user_dict={
        "email":email,
        "first_name":first_name,
        "last_name":last_name,
        "password":password
    }
    return register_user(user_dict)

def display_book_short(book):
    print(f"{book['id']} \t| {book['title'][:50].ljust(50)} | \t{book['subject']}")

def display_book_long(book):
    cover = Image(url=book['img'])
    display(cover)
    print(f'''
Title:\t {book['title']}
Author: {book['author']}
Pages:\t {book['pages']}
Subject: {book['subject']}
Summary: {book['summary']}
''')
    
    
    
class ReadingList():
    def __init__(self):
        self.reading_list=[]
        self.read_books = []
    
    def welcome_main(self):
        clear_output()
        print("Welcome to the Bookstore")
    
    def invalid(self):
        print("Invalid Username/Password Combo")
        time.sleep(2)
        
    def register_screen(self):
        print("You have successfully registered!")
    
    def goodbye(self):
        print('Goodbye!')
    
        
    def add_book(self, book):
        if book not in self.reading_list:
            self.reading_list.append(book)
    
    def add_read_book(self, book):
        if book not in self.read_books:
            self.reading_list.append(book)
            
    def show_read_books(self):
        clear_output()
        if not self.read_books:
            print("You haven't read anything you dolt.")
        for book in self.read_books:
            print(f'''
{"="*50}            
\n
Title:\t {book['title']}
Book ID: {book['id']}
Author:\t {book['author']}
Subject: {book['subject']}
Summary: {book['summary']}
\n
{"="*50}            
\n
''')
    
    def remove_book(self, book):
        self.reading_list.remove(book)
    
    def empty(self):
        self.reading_list=[]
    
    def show_book_list(self):
        clear_output()
        if not self.reading_list:
            print("Your reading list is empty")
        for book in self.reading_list:
            print(f'''
{"="*50}            
\n
Title:\t {book['title']}
Book ID: {book['id']}
Author:\t {book['author']}
Subject: {book['subject']}
Summary: {book['summary']}
\n
{"="*50}            
\n
''')
    
    def show_small_book_list(self):
        if not self.reading_list:
            print("Your reading list is empty")
        else:
            for book in self.reading_list:
                display_book_short(book)

def browse_books(books, reading_list, subject=None):
    useful = UsefulDoodads()
    while True:
        clear_output()
        print(f'''
Welcome to the Book Browser
You are viewing {subject if subject else 'all'} books
[ID] \t| {"[TITLE]".ljust(50)} | [SUBJECT]
        ''')
        if subject:
            #useful.subject()
            books=get_book_by_category(books, subject)
        for book in books:
            display_book_short(book)

        selection = input("Select your book by ID [BACK to back out]")
        if selection.lower()=='back':
            return
        elif selection.isnumeric() and int(selection) in map(lambda book: book['id'], books):
            selection=int(selection)
            while True:
                print(f'''
You Selected: {list(filter(lambda book: book['id'] == selection, books))[0]['title']}
1. Add Book To Reading List
2. View More Information
3. Go Back
4. Go To Main Menu
5. Add To 'Read' List
''')
                action = input("Action: ")
                if action =="1":
                    reading_list.add_book(list(filter(lambda book: book['id']==selection, books))[0])
                    print("As you wish")
                    time.sleep(1)
                    break
                elif action =="2":
                    clear_output()
                    display_book_long(list(filter(lambda book: book['id']==selection, books))[0])
                    input("Press Enter To Continue")
                elif action=="3":
                    break
                elif action=="4":
                    return
                elif action == "5":
                    reading_list.add_read_book(list(filter(lambda book: book['id']==selection, books))[0])
                    
        else:
            print("Invalid ID")
            time.sleep(2)
            continue
    

def main():
    reading_list = ReadingList()
    books = get_books()
    
    while True:
        reading_list.welcome_main()
        email = input("Type your email to login or Type `register` to Register ")
        if email == 'register':
            success_register=register()
            if success_register:
                reading_list.register_screen()
                continue
        elif email.lower() == "quit":
            reading_list.goodbye()
            break
        else:
            try:
                login(email)
            except:
                reading_list.invalid()
                continue
        # First Scene of our app (above)
        while True:
            clear_output()
            print("""
Welcome to the Repository            
You can:            
1. Browse All Books
2. Browse Book by Category
3. View Reading List
4. Remove Book From Reading List
5. Quit
6. Show 'Read' List

""")
            command = input("Select your Fate: ")
            if command == "1":
                browse_books(books, reading_list)
            elif command == "2":
                while True:
                    print(" | ".join(get_category_list(books)))
                    cat = input("Category: ").title()
                    if cat in get_category_list(books):
                        browse_books(books, reading_list, cat)
                        break
                    print("Invalid Category")
                    time.sleep(2)
                    continue
            elif command == "3":
                reading_list.show_book_list()
                input("Press Enter To Return")
                continue
            elif command == "4":
                while True:
                    clear_output()
                    reading_list.show_small_book_list()
                    garbage = input("What book ID would you like to remove? [BACK to back out]")
                    if garbage.lower() == "back":
                        break
                    elif garbage.isnumeric() and int(garbage) in map(lambda book: book['id'], reading_list.reading_list):
                        reading_list.remove_book(list(filter(lambda book: book['id']==int(garbage), reading_list.reading_list))[0])
                        print(f'{garbage} has been removed')
                        time.sleep(2)
                        break
                    else:
                        print(f'{garbage} is not in your collection')
                        time.sleep(2)
                        break
                continue
                    
            elif command == "5":
                reading_list.goodbye()
                #print("Goodbye")
                break
            elif command == "6":
                reading_list.show_read_books()
                input("Press Enter To Return")
                continue
            else:
                print("Invalid Selection")
                time.sleep(2)
                continue
        break
            

main()            


Welcome to the Repository            
You can:            
1. Browse All Books
2. Browse Book by Category
3. View Reading List
4. Remove Book From Reading List
5. Quit
6. Show 'Read' List




In [ ]:
### Tests

## API 
def get_category_list(books):
    return {book['subject'].title() for book in books}
get_category_list(books)

## Test for API 
def test_1_api(self):
        self.assertEqual(get_category_list({'Cooking','Entertainment','Marriage','Marriage Advice','Programming','Science','Self Help'}))
        
## UI
def display_book_short(book):
    print(f"{book['id']} \t| {book['title'][:50].ljust(50)} | \t{book['subject']}")
    
## Test for UI
 def test_1_UI(self):
        self.assertEqual(display_book_short('marriage')[ID] | [TITLE] | [SUBJECT]| 'TEACH YOUR WIFE TO BE A WIDOW' | 'marriage')


In [ ]:
cindy@crawford.com